In [ ]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')
import re
%matplotlib inline
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import make_multilabel_classification
#url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Creating SQL Engine and read a table.

In [ ]:

engine = create_engine("sqlite:////content/DisasterResponse (1).db")
# load data from database
#def load_data():
#engine = create_engine('sqlite:///DisasterResponse(1).db')
df = pd.read_sql_table('Message', engine)
    #engine = create_engine('sqlite:///DisasterResponse.db')
    #df = pd.read_sql_table('DisasterResponse', 'sqlite:///DisasterResponse.db') 
#print(df.head())
df = df.drop(['original','index'], axis=1)
#X = df.message  

#y = df[df.columns[~df.columns.isin(['message','original','genre','id'])]].values
    #return X, y

In [ ]:
X = df[['message']].values
Y = df.iloc[:,4:].values

In [ ]:
X = [i for i in X for i in i]

In [ ]:
Y1 = df.iloc[:,4].values
Y1


array([0, 0, 0, ..., 0, 0, 0])

# Tokenization sentenses.

In [ ]:
def tokenize(text):

    text = re.sub(r"[^a-zA-Z0-9]", ' ', text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        
        clean_tok = lemmatizer.lemmatize(tok, pos = 'v').lower().strip()
        
        clean_tokens.append(clean_tok)
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]

    return clean_tokens

In [ ]:
X1 = word_tokenize(X[1])

# Building a machine learning pipeline.

In [ ]:
#Build a machine learnng pipeline
#for i in model:
estimator = RandomForestClassifier()
#estimator =  MultinomialNB()
pipeline = Pipeline([
        ('transformer', Pipeline([
            ('vect', CountVectorizer(tokenizer = tokenize)),
            ('tfidf', TfidfTransformer())
            ])),
         ('clf', MultiOutputClassifier(estimator))
        ])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

In [ ]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('transformer',
                 Pipeline(steps=[('vect',
                                  CountVectorizer(tokenizer=<function tokenize at 0x7fe1c8e469e0>)),
                                 ('tfidf', TfidfTransformer())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [ ]:
Y_pred = pipeline.predict(X_test)

In [ ]:
categories = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food','shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death',  'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']
for category in categories:
# train the model using X_dtm & y
    
    # compute the testing accuracy
    
    print(Y_pred[3])

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

1 represent a classified category. I this case , third category 'offer' was classified to a santense

Creating a GridSearch for a pipeline for a better performance to choose best parameters

In [ ]:
parameters = { 
    'transformer__vect__max_features': [5000, 3000, 1000],
    'transformer__vect__ngram_range': ((1,1),(1,2)),
    'transformer__tfidf__use_idf': (True, False)
            }
cv = GridSearchCV(pipeline, param_grid = parameters)
cv.fit(X_train,Y_train)

GridSearchCV(estimator=Pipeline(steps=[('transformer',
                                        Pipeline(steps=[('vect',
                                                         CountVectorizer(tokenizer=<function tokenize at 0x7fe1c8e469e0>)),
                                                        ('tfidf',
                                                         TfidfTransformer())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'transformer__tfidf__use_idf': (True, False),
                         'transformer__vect__max_features': [5000, 3000, 1000],
                         'transformer__vect__ngram_range': ((1, 1), (1, 2))})

# Testing a pipeline performance after GridSearch.

In [ ]:
y_pred = cv.predict(X_test)
y_pred[4]

In [ ]:

categories = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food','shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death',  'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']
for category in categories:
    print(y_pred[1])
   